In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random
from tqdm import tqdm

In [ ]:
train_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.shape

In [ ]:
train_df['question_text'].isnull().sum()

In [ ]:
train_df['question_text'].isna().sum()

In [ ]:
train_df = train_df.rename({'question_text': 'question'}, axis=1)

In [ ]:
train_df.columns

In [ ]:
train_df.target.value_counts()

In [ ]:
import matplotlib.ticker as ticker

ncount = train_df.shape[0]

plt.figure(figsize=(7, 5))

ax = sns.countplot(data=train_df, x='target')
plt.title('Distribution of Questions')
plt.xlabel('Number of Axles')

# Make twin axis
ax2=ax.twinx()

# Switch so count axis is on right, frequency on left
ax2.yaxis.tick_left()
ax.yaxis.tick_right()

# Also switch the labels over
ax.yaxis.set_label_position('right')
ax2.yaxis.set_label_position('left')

ax2.set_ylabel('Frequency [%]')

for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
            ha='center', va='bottom') # set the alignment of the text

# Use a LinearLocator to ensure the correct number of ticks
ax.yaxis.set_major_locator(ticker.LinearLocator(11))

# Fix the frequency range to 0-100
ax2.set_ylim(0,100)
ax.set_ylim(0,ncount)

# And use a MultipleLocator to ensure a tick spacing of 10
ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))

# Need to turn the grid on ax2 off, otherwise the gridlines end up on top of the bars
ax2.grid(None)

In [ ]:
pd.set_option('display.max_columns', None)
insincere_qes = train_df[train_df['target'] == 1]
print(insincere_qes[:5].question)

In [ ]:
pd.set_option('display.max_columns', None)
sincere_qes = train_df[train_df['target'] == 0]
print(sincere_qes[-5:].question)

In [ ]:
train_df['question_length'] = train_df['question'].apply(len)

In [ ]:
train_df.head()

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

stop_words = stopwords.words('english')
stop_words.remove('not')
stop_words.append('would')
lemmatizer = WordNetLemmatizer()

In [ ]:
contraction_dict = {"dont": "do not", "aint": "is not", "isnt": "is not", "doesnt": "does not", "cant": "cannot", "mustnt": "must not", "hasnt": "has not", "havent": "have not", "arent": "are not", "ain't": "is not", "aren't": "are not","can't": "cannot", "‘cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "Iam": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def replace_contractions(question):
    return [contraction_dict.get(token) if (contraction_dict.get(token) != None) else token for token in question]

In [ ]:
import spacy

nlp = spacy.load("en_core_web_lg")

In [ ]:
import zipfile
import gensim

archive = zipfile.ZipFile('../input/quora-insincere-questions-classification/embeddings.zip', 'r')
#print(archive.namelist()) # print all zip content

embeddings = gensim.models.KeyedVectors.load_word2vec_format(archive.open('GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'), binary=True,limit=100000)

In [ ]:
def check_contain_digit(word):
    for i in word:
        if i.isdigit():
            return True
    return False

In [ ]:
def qes_preprocessing(qes):
    
    # Data cleaning:
    qes = re.sub(re.compile('<.*?>'), '', qes)
    qes = re.sub('[^A-Za-z0-9]+', ' ', qes)

    # Lowercase:
    qes = qes.lower()
    
    # Tokenization:
    tokens = word_tokenize(qes)

    # Contractions replacement:
    tokens = [contraction_dict.get(token) if (contraction_dict.get(token) != None) else token for token in tokens]

    # Stop words removal:
    tokens = [w for w in tokens if w not in stop_words]
    
    # Number removal:
    tokens = [w for w in tokens if any(i.isdigit() for i in w) == False]
    
    # Lemmatization:
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    return ' '.join(tokens)

In [ ]:
found_synonyms = {}

def find_synonyms(word):
    if word in found_synonyms:
        replacements = found_synonyms[word]
    else:
        replacements = embeddings.most_similar(word)
        found_synonyms[word] = replacements
    rep_index = int(len(replacements) * random.random())
    return replacements[rep_index][0].lower()

In [ ]:
def qes_preprocessing_oversampling(qes): 

    # Data cleaning:
    qes = re.sub(re.compile('<.*?>'), '', qes)
    qes = re.sub('[^A-Za-z0-9]+', ' ', qes)

    doc_ents = set([i.text.lower() for i in nlp(qes).ents])
    
    # Lowercase:
    qes = qes.lower()
    
    # Tokenization:
    tokens = word_tokenize(qes)

    # Contractions replacement:
    tokens = [contraction_dict.get(token) if (contraction_dict.get(token) != None) else token for token in tokens]

    # Stop words removal:
    tokens = [w for w in tokens if w not in stop_words]
    
    # Number removal:
    tokens = [w for w in tokens if check_contain_digit(w) == False]
    
    # Lemmatization:
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    
    final_ques = [' '.join(tokens)]
    
    for i in range(5):
        if len(tokens) > 1:
            # Number of replaced words
            no_words = int(len(tokens) * random.random())
            # Perform replacements
            for x in range(no_words):
                token_index = int(len(tokens) * random.random())
                try:
                    token = tokens[token_index]
                    # Search for token in synonyms dict
                    if token not in doc_ents:
                        synonym = find_synonyms(token)  
                        tokens[token_index] = synonym
                        final_ques.append(' '.join(tokens))
                except:
                     pass
        elif len(tokens) == 1:
            try:
                synonym = find_synonyms(token) 
                final_ques.append(synonym) 
            except:
                pass
        else:
            break

    return list(set(final_ques))

In [ ]:
embeddings.most_similar("adopt")

In [ ]:
tqdm.pandas()
test = train_df['question'][:10].progress_apply(qes_preprocessing_oversampling)

In [ ]:
test[5]

In [ ]:
train_df['question'].iloc[1]

In [ ]:
tqdm.pandas()
insincere_questions = train_df[train_df['target'] == 0]['question'].progress_apply(qes_preprocessing)

In [ ]:
X1 = pd.DataFrame({
    'target': np.zeros(insincere_questions.shape, dtype=int),
    'question': insincere_questions
})

In [ ]:
X1

In [ ]:
insincere_questions.shape

In [ ]:
tqdm.pandas()
sincere_questions = train_df[train_df['target'] == 1]['question'].progress_apply(qes_preprocessing_oversampling)

In [ ]:
sincere_questions = sincere_questions.explode('question')

In [ ]:
sincere_questions.head()

In [ ]:
sincere_questions.shape

In [ ]:
X2 = pd.DataFrame({
    'target': np.ones(sincere_questions.shape, dtype=int),
    'question': sincere_questions
})

In [ ]:
trainset = pd.concat([X1, X2])

In [ ]:
trainset

In [ ]:
trainset = trainset.dropna(how='any')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(1,4), max_df=0.9)),
                     ("clf", LogisticRegression(solver="saga", class_weight="balanced", C=0.45, max_iter=600, verbose=1, n_jobs=-1))])

X_train, X_test, y_train, y_test = train_test_split(trainset['question'], trainset.target, test_size=0.2, stratify = trainset.target.values)

In [ ]:
lr_model = pipeline.fit(X_train, y_train)

In [ ]:
def get_fscore_matrix(fitted_clf, model_name):
    print(model_name, ' :')
    
    # get classes predictions for the classification report 
    y_train_pred, y_pred = fitted_clf.predict(X_train), fitted_clf.predict(X_test)
    print(classification_report(y_test, y_pred), '\n') # target_names=y
    
    # computes probabilities keep the ones for the positive outcome only      
    print(f'F1-score = {f1_score(y_test, y_pred):.2f}')

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report

get_fscore_matrix(lr_model, 'Linear Regression')

In [ ]:
test_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
test_df.head()

In [ ]:
test_df['preprocessed'] = test_df['question_text'].apply(qes_preprocessing)

In [ ]:
predictions = lr_model.predict(test_df['preprocessed'])

In [ ]:
print(len(predictions))

In [ ]:
test_df['prediction'] = predictions
results = test_df[['qid', 'prediction']]
results.to_csv('submission.csv', index=False)
results.head()